In [28]:
%pip install --upgrade --quiet  wikipedia langchain_openai


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# brew install ffmpeg

In [3]:
from langchain_community.document_loaders import WikipediaLoader

In [4]:
docs = WikipediaLoader(query="Kevin Durant", load_max_docs=10).load()

In [5]:
from indexify import IndexifyClient
client = IndexifyClient()

In [10]:
client = IndexifyClient()
for doc in docs:
    client.add_documents(doc.page_content)

In [9]:
client.add_extraction_policy(extractor='tensorlake/chunk-extractor', name="chunker")
client.add_extraction_policy(extractor='tensorlake/minilm-l6', name="wikiembedding", content_source="chunker")

In [34]:
client.indexes()

[{'name': 'wikiembedding.embedding',
  'schema': {'dim': 384, 'distance': 'cosine'}}]

In [35]:
from indexify_langchain import IndexifyRetriever

In [36]:
params = {"name": "wikiembedding.embedding", "top_k": 50}
retriever = IndexifyRetriever(client=client, params=params)


In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [38]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [40]:
chain.invoke("When and where did KD win his championships?")

'Kevin Durant won his championships in 2017 and 2018 with the Golden State Warriors.'